In [ ]:
import requests
import re
import time 
from datetime import datetime
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import json
import os
import logging
import itertools

In [ ]:
#傻眼Ing,SQL吃不了表情文字，需要額外刪除表情文字。python竟然可以讀部分XD，厲害
import emoji

In [ ]:
#建立log，紀錄資訊
date_time=datetime.strftime(datetime.now(),'%Y-%m-%d')
path='C:/Users/lenovon/Desktop/Test/'+date_time+'.log'
if os.path.isfile(path):
    logging.basicConfig(level=logging.WARNING,#控制檯列印的日誌級別
                        filename=path,
                        filemode='a') #模式有w和a，w就是寫模式，每次都會重新寫日誌，覆蓋之前的日誌;a是append的意思
else:
    logging.basicConfig(level=logging.WARNING,#控制檯列印的日誌級別
                        filename=path,
                        filemode='w') #模式有w和a，w就是寫模式，每次都會重新寫日誌，覆蓋之前的日誌;a是append的意思

In [ ]:
#header
user_agent = [
"Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
"Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
"Mozilla/5.0 (Windows NT 10.0; WOW64; rv:38.0) Gecko/20100101 Firefox/38.0",
"Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729; InfoPath.3; rv:11.0) like Gecko",
"Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)",
"Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)",
"Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
"Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
"Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11",
"Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11",
"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Maxthon 2.0)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; TencentTraveler 4.0)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; The World)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SE 2.X MetaSr 1.0; SE 2.X MetaSr 1.0; .NET CLR 2.0.50727; SE 2.X MetaSr 1.0)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Avant Browser)",
"Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)"
]

In [ ]:
#get article url for 1 to page_num
def get_article_url(url,mode,to_page=None):
    Links=[]
    if mode ==1: #Simple_flying       
        #crawl 1~to_page url of articles
        for j in range(1,to_page+1):
            a=requests.post('https://simpleflying.com/wp-json/csco/v1/more-posts',
                    data={
                        'page':j,'action':'csco_ajax_load_more',
                        'posts_per_page':12,
                        'query_data': '{"location":"home","infinite_load":false,"query_vars":{"error":"","m":"","p":0,"post_parent":"","subpost":"","subpost_id":"","attachment":"","attachment_id":0,"name":"","pagename":"","page_id":0,"second":"","minute":"","hour":"","day":0,"monthnum":0,"year":0,"w":0,"category_name":"","tag":"","cat":"","tag_id":"","author":"","author_name":"","feed":"","tb":"","paged":0,"meta_key":"","meta_value":"","preview":"","s":"","sentence":"","title":"","fields":"","menu_order":"","embed":"","category__in":[],"category__not_in":[],"category__and":[],"post__in":[],"post__not_in":[59602,59548,59586,50517],"post_name__in":[],"tag__in":[],"tag__not_in":[],"tag__and":[],"tag_slug__in":[],"tag_slug__and":[],"post_parent__in":[],"post_parent__not_in":[],"author__in":[],"author__not_in":[],"49f47c948536c05fb3145f9e3a863558":[59602],"64c05e4b4d134438a75e15c6d2e05464":[59548,59586,50517],"ignore_sticky_posts":false,"suppress_filters":false,"cache_results":true,"update_post_term_cache":true,"lazy_load_term_meta":true,"update_post_meta_cache":true,"post_type":"","posts_per_page":12,"nopaging":false,"comments_per_page":"50","no_found_rows":false,"order":"DESC"},"in_the_loop":false,"is_single":false,"is_page":false,"is_archive":false,"is_author":false,"is_category":false,"is_tag":false,"is_tax":false,"is_home":true,"is_singular":false}'
                    },
                   headers={
                       'origin':'https://simpleflying.com',
                       'referer':'https://simpleflying.com/category/aviation-news/',
                       'content-type':'application/x-www-form-urlencoded;charset=UTF-8',
                       'User-Agent':np.random.choice(user_agent)})
            if a.status_code !=200:
                s=datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S')
                logging.error(s+':'+' post method for page whose status_code is not 200, please check.'+str(j)+' page')
            temp=a.json()
            soup=BeautifulSoup(temp['data']['content'],'lxml')
            for s in soup.find_all(class_='post-inner'):
                try:
                    temp=s.find('a').get('href')
                    Links.append(temp)
                except:
                    continue
            #delete duplicated url
            Links=[s for s in np.unique(Links)]
            time.sleep(4)
    elif mode ==2: #Reuters
        a=requests.get(url,timeout=20,
                   headers={'User-Agent':np.random.choice(user_agent)})
        if a.status_code !=200:
            s=datetime.strftime(datetime.now(),'%Y-%m-%d %H:%M:%S')
            logging.error(s+':'+' status_code is not 200, url is : '+url)
        soup=BeautifulSoup(a.text,'lxml')
        try:
            Links=['https://www.reuters.com/'+s.find('a').get('href') for s in soup.find('div',class_='column1 col col-10').find_next('div',class_='news-headline-list').find_all(class_='story-content')]
        except:
            print('Can not find any article url for this url,please check.')
            Links=[]
    else: # FlightGlobal
        a=requests.get(url,timeout=20,
                   headers={'User-Agent':np.random.choice(user_agent)})
        if a.status_code !=200:
            s=datetime.strftime(datetime.now(),'%Y-%m-%d %H:%M:%S')
            logging.error(s+':'+' status_code is not 200, url is : '+url)
        soup=BeautifulSoup(a.text,'lxml')
        try:
            Links=[s.find('a').get('href') for s in soup.find_all('h3')]
        except:
            print('Can not find any article url for thiss url,please check.')
            Links=[]
    return(Links)

In [ ]:
url2=['https://www.reuters.com/news/archive/businessnews?view=page&page=1&pageSize=10',
      'https://www.reuters.com/news/archive/marketsnews?view=page&page=1&pageSize=10',
      'https://www.reuters.com/news/archive/worldnews?view=page&page=1&pageSize=10',
      'https://www.reuters.com/news/archive/politicsnews?view=page&page=1&pageSize=10',
      'https://www.reuters.com/news/archive/mcbreakingviews?view=page&page=1&pageSize=10',
      'https://www.reuters.com/news/archive/personalfinance?view=page&page=1&pageSize=10']
url3=['https://www.flightglobal.com/236.more?page=1']

In [ ]:
try:
    Links1=get_article_url(to_page=3,url=None,mode=1)
    Links2=[]
    for u in url2:
        uu=get_article_url(url=u,mode=2)
        for s in uu:
            Links2.append(s)
        time.sleep(2)
    Links3=[]
    for u in url3:
        uu=get_article_url(url=u,mode=3)
        for s in uu:
            Links3.append(s)
except:
    logging.error('Function:get_article_url is wrong,please check.',exc_info=True)

In [ ]:
#crawl any useful information of article
def crawl_info(article_url,index,mode):
    if mode ==1: #Simple_Flying
        a=requests.get(article_url,timeout=20)
        if a.status_code !=200:
            s=datetime.datetime.strftime(datetime.datetime.now(),'%Y-%m-%d %H:%M:%S')
            logging.error(s+':'+' some status_code of url is not 200, please check.\n'+article_url)
        soup=BeautifulSoup(a.text,'lxml')
        try:
            title=soup.find(class_='entry-header-inner').find('h1').text
        except:
            title=' '
        try:
            author=soup.find(class_='entry-header-inner').find(class_='author').text
        except:
            author=' '
        #SQL 的麻煩資料時間格式
        try:
            dt_publish=soup.find(class_='entry-header-inner').find(class_='meta-date').text
            dt_publish=datetime.strftime(datetime.strptime(dt_publish,'%B %d, %Y'),'%Y/%m/%d')
        except:
            dt_publish='NULL'
        try:
            temp=soup.find(class_='wprt-container').find_all('p')
            temp_c=[s.text for s in temp]
            #ignore 1 and find first position of subtitle in content
            sub_header=[s.find_previous('h3') for s in temp]
            sub_header_logical=pd.DataFrame(sub_header,columns=['V1']).duplicated().tolist()
            if len(temp_c) != len(sub_header):
                raise RuntimeError('Error for length of content not equal to length of boolean of sub_header')
            else:
                content=''
                for i in range(0,len(temp_c)):
                    if not sub_header_logical[i] and i !=0:
                        content=content+'\n'+sub_header[i].text+'\n'
                        content=content+temp_c[i]
                    else:
                        content=content+temp_c[i]
                if content == '':
                    content=' '
                else:
                    content=re.sub(string=content,pattern='\xa0',repl=' ')
                    content=emoji.demojize(content)
        except:
            content=' '
        try:
            img=''
            temp_img=[s.find('img').get('src') for s in soup.find_all('figure')]
            for i,s in enumerate(temp_img,start=1):
                if i != len(temp_img):
                    img=img+s+';'
                else:
                    img=img+s
            if img == '':
                img=' '
        except:
            img=' '
        return pd.DataFrame({'title':title,'author':author,'category':' ','url':article_url,
                             'content':content,'dt_publish':dt_publish,'img':img,
                            'Source':'Simple_Flying'},
                           columns=['author','dt_publish','title','content','url','img','Source','category'],index=[index])
    elif mode ==2: #Reuters
        a=requests.get(article_url,
                       headers={'User-Agent':np.random.choice(user_agent)},
                       timeout=20)
        if a.status_code !=200:
            s=datetime.strftime(datetime.now(),'%Y-%m-%d %H:%M:%S')
            logging.error(s+':'+' status_code is not 200,url of article is : '+article_url)
        soup=BeautifulSoup(a.text,'lxml')
        temp=soup.find(class_='ArticleHeader_content-container')
        #category
        try:
            category=temp.find('a').text
        except:
            print('Can not find category of article,please check.')
            category=' '
        #dt_publish only date.
        try:
            dt_publish=re.search(string=temp.find(class_='ArticleHeader_date').text,
                                 pattern='(.+\d+) /').group(1)
        # date contain hours and minute
        # dt_publish=re.search(string=temp.find(class_='ArticleHeader_date').text,
        #                      pattern='.+[AP]*M').group(0)
            dt_publish=datetime.strftime(datetime.strptime(dt_publish,'%B %d, %Y'),'%Y/%m/%d')
        except:
            print('Can not find publish time of article,please check.')
            dt_publish=' '
        #title
        try:
            title=temp.find('h1').text
        except:
            print('Can not find title of article,please check')
            title=' '
        #author
        try:
            author=soup.find('div',class_='BylineBar_byline').text
        except:
            print('Can not find author of article,please check')
            author=' '
        #content
        try:
            content=''
            for s in soup.find(class_='StandardArticleBody_body').find_all('p'):
                content=content+s.text+'\n'
            content=re.sub(string=content,pattern='\xa0',repl='')
            content=re.sub(string=content,pattern='\n$',repl='')
        except:
            content=' '
            print('Can not find content of article,please check')
        #lazy image
        try:
            img=soup.find('div',class_='LazyImage_container').find('img').get('src')
            img=re.sub(string=img,pattern='w=20',repl='w=1280')
        except:
            print('Can not find image of article,please check')
            img=' '
        return(pd.DataFrame({'title':title,
                              'author':author,
                              'category':category,
                              'url':article_url,
                              'content':content,
                              'dt_publish':dt_publish,
                              'img':img,
                              'Source':'Reuters'},
                            columns=['author','dt_publish','category','title','content','url','img','Source'],
                            index=[index]))
    else: #FlightGlobal
    #flightgobal防護機制
    #需要裝成瀏覽器，否則直接拒絕
    #有些需要登入才給你看的文章(可能是他們認為重要的或隨機選文章)
        a=requests.get(article_url,timeout=20,
                       headers={'User-Agent':np.random.choice(user_agent),
                                'Host':'www.flightglobal.com'})
        if a.status_code !=200:
            s=datetime.strftime(datetime.now(),'%Y-%m-%d %H:%M:%S')
            logging.error(s+':'+' status_code is not 200,url of article is : '+article_url)
        soup=BeautifulSoup(a.text,'lxml')
        temp=soup.find('div',class_='container titleByline')
        #category
        try:
            category=temp.find('div',class_='storyPrimaryNav').text
        except:
            print('Can not find category of article,please check.')
            category=' '
        #dt_publish only date.
        try:
            dt_publish=datetime.strptime(temp.find(class_='date').text,'%Y-%m-%dT%H:%M:%S%z')
            dt_publish=datetime.strftime(dt_publish,'%Y/%m/%d')
        except:
            print('Can not find publish time of article,please check.')
            dt_publish=' '
        #title
        try:
            title=temp.find('h1').text
        except:
            print('Can not find title of article,please check')
            title=' '
        #author
        try:
            author=temp.find(class_='author').text
            author=re.sub(pattern='^By |^by ',repl='',string=author)
        except:
            print('Can not find author of article,please check')
            author=' '
        #content
        try:
            content=''
            for s in soup.find(class_='storytext').find_all('p',recursive=False):
                content=content+s.text+'\n'
            content=re.sub(string=content,pattern='\xa0',repl='')
            content=re.sub(string=content,pattern='\n$',repl='')
        except:
            content=' '
            print('Can not find content of article,please check')
        #lazy image
        try:
            img=soup.find(class_='storytext').find(class_='image_size_full').find('img').get('data-src')
        except:
            print('Can not find image of article,please check')
            img=' '
        return(pd.DataFrame({'title':title,
                              'author':author,
                              'category':category,
                              'url':article_url,
                              'content':content,
                              'dt_publish':dt_publish,
                              'img':img,
                              'Source':'FlightGlobal'},
                            columns=['author','dt_publish','category','title','content','url','img','Source'],
                            index=[index]))


In [ ]:
try:
    D=pd.DataFrame(columns=['author','dt_publish','title','category','content','url','img','Source'])
    ind=0
    for s in Links1:
        D=pd.concat([D,crawl_info(article_url=s,index=ind,mode=1)])
        ind=ind+1
    for s in Links2:
        D=pd.concat([D,crawl_info(article_url=s,index=ind,mode=2)])
        ind=ind+1
    for s in Links3:
        D=pd.concat([D,crawl_info(article_url=s,index=ind,mode=3)])
        ind=ind+1
except:
    logging.error('Function:crawl_info is wrong,please check.',exc_info=True)

In [ ]:
D_new=D

In [ ]:
D_new=D_new[~D_new.duplicated(subset=['url'])]
D_new=D_new.sort_values(by=['dt_publish'],ascending=True)
D_new=D_new.reset_index(drop=True)

In [ ]:
#每天寫入SQL資料庫
import pymysql
import MySQLdb
from sqlalchemy import create_engine
import os

In [ ]:
conn = pymysql.connect(
    host='ip',
    port='port',
    user='username',
    password='password',
    db='dbname')
engine = create_engine('mysql+mysqldb://username:password@ip:port/dbname?charset=utf8mb4')
all_table=pd.read_sql_query('Show tables',conn)

In [ ]:
cursor=conn.cursor()

In [ ]:
all_table

In [ ]:
#insert new data in table news
table_name='news'
try:
    #有檔案
    #讀取舊檔
    D_old=pd.read_sql_table(table_name,con=engine)
    #修改從sql讀取出來的datetime格式
    D_old['dt_publish']=D_old['dt_publish'].astype('str')
    #舉例:將2010-01-01轉變成2010/01/01 
    D_old['dt_publish']=[re.sub(string=s,pattern='-',repl='/') for s in D_old['dt_publish']]
    #找尋全新資料(以作者、發文時間、標題合起來當作primary key)
    pos=pd.merge(D_new,D_old,on=['title','author','dt_publish'],how='left',indicator=True)['_merge'] == 'left_only'
    pos=pos.values.tolist()
    if sum(pos) == 0:
        print('No new data.')
    else:
        D_new=D_new.iloc[pos,:]
        #去除從不同地方抓到同樣的文章
        D_new=D_new[~D_new.duplicated(subset=['title','author','dt_publish'])]
        D_new=D_new.sort_values(by=['dt_publish'],ascending=True)
        D_new=D_new.reset_index(drop=True)
        #存入新資料
        for r in range(0,D_new.shape[0]):
            cursor.execute('insert into news(title,content,url,dt_publish,author,Source,category) values (%s,%s,%s,%s,%s,%s,%s)',
                          (D_new.iloc[r,6],D_new.iloc[r,3],D_new.iloc[r,7],
                           D_new.iloc[r,4],D_new.iloc[r,1],D_new.iloc[r,0],
                           D_new.iloc[r,2]))
        conn.commit()    
except:
    logging.warning('File to SQL cannot be done due to:',exc_info=True)

In [ ]:
import wordsegment as ws

In [ ]:
def keyword_search(txt,keyword):
    #preprocess
    keyword_temp=[ws.clean(k) for k in keyword]
    pat=''
    for i,k in enumerate(keyword_temp):
        if len(keyword_temp)>1:
            if i not in[len(keyword_temp)-1]:
                pat=pat+k+'|'
            else:
                pat=pat+k
        else:
            pat=k
    txt=ws.clean(txt)
    #find if keyword in article or not
    find_key=list(set(re.findall(string=txt,pattern=pat)))
    #which keyword in the ariticle and store boolean logic
    result=[s in find_key for s in keyword_temp]
    return result

In [ ]:
#data contains nid
def restructure_content_key(dat):
    #從SQL讀取keyword表格
    try:
        key_w=pd.read_sql_table('keyword',con=engine)
        keyword=key_w['keyword'].values.tolist()
    except:
        logging.error('Can not read keyword table from sql ,please check')
    Final=pd.DataFrame(columns=['nid','kid'])
    for i in range(0,dat.shape[0]):
        nid=dat['nid'].iloc[i]
        txt=dat['content'].iloc[i]
        temp=keyword_search(txt=txt,keyword=keyword)
        if sum(temp) !=0:
            Final=pd.concat([Final,
                             pd.DataFrame({'nid':nid,'kid':key_w['kid'].iloc[temp].values})])
        else:
            continue
    Final=Final.reset_index(drop=True)
    return(Final)

In [ ]:
#find relation of article and keyword.And insert into table news_keyword
table_name='news_dealt'
try:
    #有檔案
    #讀取舊檔
    Relate_old=pd.read_sql_table(table_name,con=engine)
    #找尋新資料
    D_new=pd.read_sql_query('select * from news',engine)
    temp=list(set(D_new['nid'].values.tolist())-set(Relate_old['nid'].values.astype('int').tolist()))
    if len(temp) == 0:
        print('Do not need to take any action because no new article')
    else:
        pos=[i for i,s in enumerate(D_new.nid) if s in temp]
        D_new=D_new.iloc[pos,:]
        D_new=D_new.reset_index(drop=True)
        #紀錄資料以處理
        for r in range(0,D_new.shape[0]):
            cursor.execute('insert into news_dealt(nid) values (%s)',
                          (str(D_new['nid'].iloc[r])))
        conn.commit()
        #存入新資料
        D_new=restructure_content_key(D_new)
        if D_new.shape[0] == 0:
            print('Do not need to take any action because all keywords not in all articles')
        else:
            for r in range(0,D_new.shape[0]):
                cursor.execute('insert into news_keyword(nid,kid) values (%s,%s)',
                              (D_new.iloc[r,0],D_new.iloc[r,1]))
            conn.commit()
    conn.close()
except:
    logging.warning('News_keyword to SQL cannot be done due to:',exc_info=True)
    conn.close()

In [ ]:
pd.read_sql_table('news_keyword',engine)

In [ ]:
pd.read_sql_table('news',engine)